Name:Vamshiteja Kaspe     
     Znumber:z23733757           

     google collab link:https://colab.research.google.com/drive/1_WZXBzSd4eGrB3Jt8oexR2snyXLWLfb5?usp=sharing

In [ ]:
#import libraries
import numpy as np

## Establish the Environment

In [ ]:
# Define the dimensions of the delivery area (i.e., its states)
delivery_area_rows = 10
delivery_area_columns = 10

# Create a 3D numpy array to store the current Q-values for each state and action pair: Q(s, a)
# The array comprises 10 rows and 10 columns (to match the shape of the delivery area), along with a third "action" dimension.
# The "action" dimension consists of 4 layers that help us track the Q-values for each possible action in each state.
# The initial value of each (state, action) pair is set to 0.
q_values = np.zeros((delivery_area_rows, delivery_area_columns, 4))


#### Actions

In [ ]:
#define actions
#numeric action codes: 0 = up, 1 = right, 2 = down, 3 = left
actions = ['up', 'right', 'down', 'left']

#### Rewards

In [ ]:
# Create a 2D numpy array to store the rewards for each state.
# The array consists of 10 rows and 10 columns (matching the shape of the delivery area),
# with each value initialized to -100.
rewards = np.full((delivery_area_rows, delivery_area_columns), -100.)
rewards[0, 5] = 100.  # Set the reward for the packaging area (i.e., the goal) to 100

# Define aisle locations (i.e., white squares) for rows 1 through 9
aisles = {}  # Store locations in a dictionary
aisles[1] = [i for i in range(1, 10)]
aisles[2] = [1, 7, 9]
aisles[3] = [i for i in range(1, 8)]
aisles[3].append(9)
aisles[4] = [3, 7]
aisles[5] = [i for i in range(10)]
aisles[6] = [5]
aisles[7] = [i for i in range(1, 10)]
aisles[8] = [3, 7]
aisles[9] = [i for i in range(10)]

# Set the rewards for all aisle locations (i.e., white squares)
for row_index in range(1, 10):
    for column_index in aisles[row_index]:
        rewards[row_index, column_index] = -1.

# Print rewards matrix
for row in rewards:
    print(row)


[-100. -100. -100. -100. -100.  100. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
[-100.   -1. -100. -100. -100. -100. -100.   -1. -100.   -1.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1. -100.   -1.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-100. -100. -100. -100. -100.   -1. -100. -100. -100. -100.]
[-100.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.   -1.]
[-100. -100. -100.   -1. -100. -100. -100.   -1. -100. -100.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


#### Define Helper Functions

In [ ]:
# Define a function to determine if the specified location is a terminal state
def is_terminal_state(current_row_index, current_column_index):
    # If the reward for this location is -1, it is not a terminal state (i.e., a 'white square')
    if rewards[current_row_index, current_column_index] == -1.:
        return False
    else:
        return True

# Define a function to choose a random, non-terminal starting location
def get_starting_location():
    current_row_index = np.random.randint(delivery_area_rows)
    current_column_index = np.random.randint(delivery_area_columns)
    # Continue choosing random row and column indexes until a non-terminal state is identified
    while is_terminal_state(current_row_index, current_column_index):
        current_row_index = np.random.randint(delivery_area_rows)
        current_column_index = np.random.randint(delivery_area_columns)
    return current_row_index, current_column_index

# Define an epsilon-greedy algorithm to choose the next action (i.e., where to move next)
def get_next_action(current_row_index, current_column_index, epsilon):
    if np.random.random() < epsilon:
        return np.argmax(q_values[current_row_index, current_column_index])
    else:
        return np.random.randint(4)

# Define a function to get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
    new_row_index = current_row_index
    new_column_index = current_column_index
    if actions[action_index] == 'up' and current_row_index > 0:
        new_row_index -= 1
    elif actions[action_index] == 'right' and current_column_index < delivery_area_columns - 1:
        new_column_index += 1
    elif actions[action_index] == 'down' and current_row_index < delivery_area_rows - 1:
        new_row_index += 1
    elif actions[action_index] == 'left' and current_column_index > 0:
        new_column_index -= 1
    return new_row_index, new_column_index

# Define a function to get the shortest path between any location within the delivery area that
# the drone is allowed to travel and the item packaging location.
def get_shortest_path(start_row_index, start_column_index):
    if is_terminal_state(start_row_index, start_column_index):
        return []
    else:
        current_row_index, current_column_index = start_row_index, start_column_index
        shortest_path = []
        shortest_path.append([current_row_index, current_column_index])
        while not is_terminal_state(current_row_index, current_column_index):
            action_index = get_next_action(current_row_index, current_column_index, 1.)
            current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
            shortest_path.append([current_row_index, current_column_index])
        return shortest_path


#### Train the AI Agent using Q-Learning

In [ ]:
# Define training parameters
exploration_rate = 0.9  # The percentage of time when we explore (choose a random action) instead of exploiting the best action
discount_factor = 0.9  # Discount factor for future rewards
learning_rate = 0.9  # The rate at which the AI agent learns

# Run through 1000 training episodes
for episode in range(1000):
    # Get the starting location for this episode
    row_index, column_index = get_starting_location()

    # Continue taking actions (i.e., moving) until reaching a terminal state
    # (i.e., reaching the item packaging area or crashing into an item storage location)
    while not is_terminal_state(row_index, column_index):
        # Choose which action to take (i.e., where to move next)
        action_index = get_next_action(row_index, column_index, exploration_rate)

        # Perform the chosen action and transition to the next state (i.e., move to the next location)
        old_row_index, old_column_index = row_index, column_index  # Store the old row and column indexes
        row_index, column_index = get_next_location(row_index, column_index, action_index)

        # Receive the reward for moving to the new state and calculate the temporal difference
        reward = rewards[row_index, column_index]
        old_q_value = q_values[old_row_index, old_column_index, action_index]
        temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

        # Update the Q-value for the previous state and action pair
        new_q_value = old_q_value + (learning_rate * temporal_difference)
        q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training complete!')


Training complete!


## Retrieve Optimal Routes

In [ ]:
# Display a few shortest paths
print(get_shortest_path(3, 9))  # Starting at row 2, column 8
print(get_shortest_path(5, 0))  # Starting at row 4, column 1
print(get_shortest_path(9, 5))  # Starting at row 8, column 7


[[3, 9], [2, 9], [1, 9], [1, 8], [1, 7], [1, 6], [1, 5], [0, 5]]
[[5, 0], [5, 1], [5, 2], [5, 3], [4, 3], [3, 3], [3, 2], [3, 1], [2, 1], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [0, 5]]
[[9, 5], [9, 6], [9, 7], [8, 7], [7, 7], [7, 6], [7, 5], [6, 5], [5, 5], [5, 6], [5, 7], [4, 7], [3, 7], [2, 7], [1, 7], [1, 6], [1, 5], [0, 5]]


In [ ]:
# Display an example of a reversed shortest path
example_path = get_shortest_path(5, 2)  # Go to row 3, column 7
example_path.reverse()
print(example_path)


[[0, 5], [1, 5], [1, 4], [1, 3], [1, 2], [1, 1], [2, 1], [3, 1], [3, 2], [3, 3], [4, 3], [5, 3], [5, 2]]
